# Analysis of Air Quality in India

This project aims at looking at a dataset of air-quality, visualizing it and finding some inferences from it. The data set used in this this project was downloaded from [Kaggle.com](https://www.kaggle.com/shrutibhargava94/india-air-quality-data).

Python libraries used for this project are
- Pandas        (for reading, parsing and aggregating data)
- NumPy         (for mathematical computation)
- Matplotlib    (for visualizing data)
- Seaborn       (for visualizing data)
- Jovian        (for saving the notebook to [Jovian.ml](https://jovian.ml))

As a first step, let's upload our Jupyter notebook to [Jovian.ml](https://jovian.ml).

In [ ]:
import jovian

In [ ]:
project_name = "analysis-of-air-quality-in-india"
file_name = "main.ipynb"

In [ ]:
def save():
    jovian.commit(project=project_name, filename=file_name, environment='pip', privacy='secret')

## Data Preparation and Cleaning

TODO

In [ ]:
save()

## Exploratory Analysis and Visualization

TODO

In [ ]:
save()

## Asking and Answering Questions

TODO

In [ ]:
save()

## Inferences and Conclusion

TODO

In [ ]:
save()

## References and Future Work

TODO

In [ ]:
save()